In [120]:
import pandas as pd
import numpy as np
import datetime
import holidays
pd.options.display.float_format = '{:.2f}'.format

In [121]:
df = pd.read_csv("data/bikes.csv", index_col=0)

<p> Eliminamos la columna index al tratarse de un índice </p>

In [122]:
df.drop(['instant'], axis= 1, inplace= True)

<p> Cambiamos la columna dteday a datetime </p>

In [123]:
df["dteday"] = pd.to_datetime(df["dteday"], format='%d-%m-%Y')

In [124]:
df.head(15)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2018-01-01,spring,0,1,0,6,0,2,14.11,18.18,80.58,10.75,331,654,985
1,2018-01-02,spring,0,1,0,0,0,2,14.90,17.69,69.61,16.65,131,670,801
2,2018-01-03,spring,0,1,0,1,1,1,8.05,9.47,43.73,16.64,120,1229,1349
3,2018-01-04,spring,0,1,0,2,1,1,8.20,10.61,59.04,10.74,108,1454,1562
4,2018-01-05,spring,0,1,0,3,1,1,9.31,11.46,43.70,12.52,82,1518,1600
5,2018-01-06,spring,0,1,0,4,1,1,8.38,11.66,51.83,6.00,88,1518,1606
6,2018-01-07,spring,0,1,0,5,1,2,8.06,10.44,49.87,11.30,148,1362,1510
7,2018-01-08,spring,0,1,0,6,0,2,6.76,8.11,53.58,17.88,68,891,959
8,2018-01-09,spring,0,1,0,0,0,1,5.67,5.81,43.42,24.25,54,768,822
9,2018-01-10,spring,0,1,0,1,1,1,6.18,7.54,48.29,14.96,41,1280,1321


Recategorizamos la columna "yr"

In [125]:
df['yr'] = df['dteday'].apply(lambda x: x.year)

Recategorizamos la columna "mnth"

In [126]:
df['mnth'] = df['dteday'].apply(lambda x: x.strftime("%b"))

In [127]:
meses = {
    "Jan": "ene",
    "Feb": "feb",
    "Mar": "mar",
    "Apr": "abr",
    "May": "may",
    "Jun": "jun",
    "Jul": "jul",
    "Aug": "ago",
    "Sep": "sep",
    "Oct": "oct",
    "Nov": "nov",
    "Dec": "dic"
}

df['mnth'] = df['mnth'].replace(meses)

Recategorizamos la columna "weekday"

In [128]:
df['weekday'] = df['dteday'].apply(lambda x: x.strftime("%a"))

In [129]:
dias_semana = {
    "Mon": "lun",
    "Tue": "mar",
    "Wed": "mie",
    "Thu": "jue",
    "Fri": "vie",
    "Sat": "sab",
    "Sun": "dom"
}

df['weekday'] = df['weekday'].replace(dias_semana)

Recategorizamos la columna "season"

In [130]:
def estacion(fecha):
    
    #Establecemos los periodos de cada estación:
    primavera_inicio = pd.Timestamp(fecha.year, 3, 20)
    primavera_fin = pd.Timestamp(fecha.year, 6, 20)
    verano_inicio = pd.Timestamp(fecha.year, 6, 21)
    verano_fin = pd.Timestamp(fecha.year, 9, 22)
    otono_inicio = pd.Timestamp(fecha.year, 9, 23)
    otono_fin = pd.Timestamp(fecha.year, 12, 20)
    
    #hacemos la conversión de fecha a estación
    if primavera_inicio <= fecha <= primavera_fin:
        return "primavera"
    elif verano_inicio <= fecha <= verano_fin:
        return "verano"
    elif otono_inicio <= fecha <= otono_fin:
        return "otoño"
    else:
        return "invierno"

# Hacemos el apply:
df["season"] = df["dteday"].apply(estacion)

Recategorizamos "holiday"

In [131]:
lista_festivos = []

fiestas_dc=holidays.US(subdiv='DC', years=[2018, 2019])

for fecha, name in sorted(fiestas_dc.items()):
    lista_festivos.append(str(fecha))


def holidays(fecha):
    
    if str(fecha).split(" ")[0] in lista_festivos:
        return "festivo"
    
    else:
        return "no festivo"

In [132]:
df["holiday"] = df["dteday"].apply(holidays)

In [133]:
df.groupby("yr")["holiday"].value_counts()

yr    holiday   
2018  no festivo    353
      festivo        12
2019  no festivo    354
      festivo        11
Name: count, dtype: int64

Recategorizamos "workingday"

In [134]:
def dias_laborables(row):
    if row["holiday"] == "festivo" or row["weekday"] in ["sab", "dom"]:
        return "no laborable"
    else:
        return "laborable"

In [135]:
df["workingday"] = df.apply(dias_laborables, axis=1)

Regaregorizamos "weathersit"

In [136]:
mapeo_weather = {
    1: 'buen tiempo',
    2: 'nublado',
    3: 'ligeramente mal tiempo',
    4: 'mal tiempo'
}
df['weathersit'] = df['weathersit'].replace(mapeo_weather)

Cambiamos el nombre a las columnas:

In [137]:
# Cambio nombre columnas
df.rename(columns = {"dteday": "fecha","season": "estacion", "yr": "año", "mnth": "mes", "holiday": "festivo", "weekday": "dia","workingday": "laborable","weathersit": "meteo",
                     "temp": "temperatura","atemp": "sensacion_termica","hum": "humedad","windspeed": "velo_viento","casual": "usu_casuales", "registered": "usu_registrados", 
                     "cnt": "total_alquiler"}, inplace = True)

In [138]:
df.sample(10)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_casuales,usu_registrados,total_alquiler
22,2018-01-23,invierno,2018,ene,no festivo,mar,laborable,buen tiempo,3.96,4.94,43.65,16.52,150,836,986
590,2019-08-14,verano,2019,ago,no festivo,mie,laborable,buen tiempo,29.79,33.84,68.67,11.33,1128,5656,6784
663,2019-10-26,otoño,2019,oct,no festivo,sab,no laborable,nublado,22.38,26.14,80.71,8.88,1182,6262,7444
616,2019-09-09,verano,2019,sep,no festivo,lun,laborable,buen tiempo,25.01,28.95,54.75,15.04,2570,5657,8227
132,2018-05-13,primavera,2018,may,no festivo,dom,no laborable,nublado,21.01,24.71,86.33,12.04,692,3413,4105
289,2018-10-17,otoño,2018,oct,no festivo,mie,laborable,buen tiempo,21.90,25.54,57.96,11.75,713,3857,4570
401,2019-02-06,invierno,2019,feb,no festivo,mie,laborable,buen tiempo,11.59,14.82,62.22,10.30,206,3578,3784
632,2019-09-25,otoño,2019,sep,no festivo,mie,laborable,buen tiempo,22.55,27.21,57.00,15.83,845,6693,7538
380,2019-01-16,invierno,2019,ene,no festivo,mie,laborable,buen tiempo,7.79,9.53,52.25,15.50,217,2081,2298
186,2018-07-06,verano,2018,jul,no festivo,vie,laborable,buen tiempo,29.52,34.28,74.33,10.04,784,3845,4629


Cambiamos el tipo de "año":

In [143]:
df["año"] = pd.Categorical(df["año"])

In [144]:
df.dtypes

fecha                datetime64[ns]
estacion                     object
año                        category
mes                          object
festivo                      object
dia                          object
laborable                    object
meteo                        object
temperatura                 float64
sensacion_termica           float64
humedad                     float64
velo_viento                 float64
usu_casuales                  int64
usu_registrados               int64
total_alquiler                int64
dtype: object

<p> Guardar los cambios en este archivo al final de la sesión </p>

In [145]:
df.to_pickle("data/bikes_limpieza.pkl")